In [1]:
# Importaciones necesarias
import os
import time
import pandas as pd
import threading
import concurrent.futures
from tqdm import tqdm
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# Cargar variables de entorno (coloca tu API key en un archivo .env)
load_dotenv("template.env")
api_key = os.getenv("OPENAI_API_KEY")

# Inicializar el cliente de OpenAI
client = OpenAI(api_key=api_key)

# Configuración
NUMERO_DIRECCIONES = 1000  # Puedes ajustar este número
CIUDAD = "Madrid"
MODELO = "gpt-4o"  # o "gpt-3.5-turbo" según tu acceso
OUTPUT_FILE = f"direcciones_{CIUDAD.lower()}_1000.xlsx"
MAX_THREADS = 5  # Número máximo de hilos concurrentes

# Prompts para solicitar direcciones - volvemos al original
PROMPT_CIUDAD = f"""Dame una dirección aleatoria en {CIUDAD}. 
Genera una dirección completa, incluyendo número, y asegúrate de que exista. 
Devuelve solo la dirección completa, sin texto adicional. Intenta que la dirección sea real y válida, además incluye calles no tan conocidas o menos comunes para evitar repeticiones."""

# Lock para acceso seguro a recursos compartidos
thread_lock = threading.Lock()

# Función para llamar a la API de OpenAI con reintentos
def get_completion(prompt, max_retries=5, retry_delay=5):
    # Crear un cliente local para cada thread para evitar problemas de concurrencia
    local_client = OpenAI(api_key=api_key)
    
    for attempt in range(max_retries):
        try:
            response = local_client.chat.completions.create(
                model=MODELO,
                messages=[
                    {"role": "system", "content": "Eres un asistente útil que genera direcciones reales existentes."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.9,  # Mayor temperatura para más aleatoriedad
                max_tokens=100
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            with thread_lock:
                print(f"Error en intento {attempt + 1}/{max_retries}: {e}")
            if attempt < max_retries - 1:
                with thread_lock:
                    print(f"Reintentando en {retry_delay} segundos...")
                time.sleep(retry_delay)
            else:
                with thread_lock:
                    print("Número máximo de intentos alcanzado. Fallando.")
                return None

# Función para procesar una dirección individual
def procesar_direccion(id):
    direccion = get_completion(PROMPT_CIUDAD)
    
    if direccion:
        return {
            "ID": id + 1,
            "Dirección": direccion,
            "Ciudad": CIUDAD
        }
    return None

# Recopilar direcciones con paralelismo
print(f"Solicitando {NUMERO_DIRECCIONES} direcciones aleatorias en {CIUDAD} usando {MAX_THREADS} hilos...")

# Lista para almacenar los datos
datos_direcciones = []
errores = 0

# Usar ThreadPoolExecutor para paralelizar las solicitudes
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    # Crear tareas futuras
    futures = [executor.submit(procesar_direccion, i) for i in range(NUMERO_DIRECCIONES)]
    
    # Procesar los resultados a medida que se completan
    for future in tqdm(concurrent.futures.as_completed(futures), total=NUMERO_DIRECCIONES):
        resultado = future.result()
        if resultado:
            with thread_lock:
                datos_direcciones.append(resultado)
        else:
            with thread_lock:
                errores += 1

# Crear DataFrame y guardar como Excel
df = pd.DataFrame(datos_direcciones)

# Ordenar por ID para mantener el orden original
df = df.sort_values(by='ID')

# Guardar como Excel
df.to_excel(OUTPUT_FILE, index=False)

print(f"\nProceso completado:")
print(f"✅ Se han guardado {len(datos_direcciones)} direcciones en {OUTPUT_FILE}")
if errores > 0:
    print(f"❌ Fallaron {errores} solicitudes")

# Mostrar algunas direcciones de ejemplo
print("\nEjemplos de direcciones obtenidas:")
for i in range(min(5, len(datos_direcciones))):
    try:
        print(f"  {i+1}. {df.iloc[i]['Dirección']}")
    except:
        # En caso de error al mostrar ejemplos
        pass

Solicitando 1000 direcciones aleatorias en Madrid usando 5 hilos...


100%|██████████| 1000/1000 [03:37<00:00,  4.60it/s]



Proceso completado:
✅ Se han guardado 1000 direcciones en direcciones_madrid_1000.xlsx

Ejemplos de direcciones obtenidas:
  1. Calle de Luis Mitjans, 37, 28007 Madrid, España.
  2. Calle de la Ribera del Manzanares, 123, 28008 Madrid, España.
  3. Calle de la Ribera del Loira, 50, 28042 Madrid, España.
  4. Calle de La Sombrerería, 12, 28012 Madrid, España.
  5. Calle de Serafín Olave, 12, 28041 Madrid, España.
